# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [76]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [77]:
# Navigate to folder where data is store
!pwd

/Users/jeremysilva/Desktop/Springboard/Guided_Capstone/data


In [78]:
cd data

[Errno 2] No such file or directory: 'data'
/Users/jeremysilva/Desktop/Springboard/Guided_Capstone/data


In [79]:
# Load the step3 output into a dataframe called ski_data
ski_data= pd.read_csv('step3_output.csv', index_col=[0])
ski_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 27 columns):
Name                 330 non-null object
state                330 non-null object
summit_elev          330 non-null float64
vertical_drop        330 non-null float64
base_elev            330 non-null float64
trams                330 non-null float64
fastEight            330 non-null float64
fastSixes            330 non-null float64
fastQuads            330 non-null float64
quad                 330 non-null float64
triple               330 non-null float64
double               330 non-null float64
surface              330 non-null float64
total_chairs         330 non-null float64
Runs                 330 non-null float64
TerrainParks         330 non-null float64
LongestRun_mi        330 non-null float64
SkiableTerrain_ac    330 non-null float64
Snow Making_ac       330 non-null float64
daysOpenLastYear     330 non-null float64
yearsOpen            330 non-null float64
averageSnowfall

In [80]:
ski_data.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,K-Means Clusters
0,Alyeska Resort,Alaska,3939.0,962.250,250.0,0.0,0.0,0.0,0.0,0.0,...,200.0,113.0,115.103943,60.0,158.0,65.000000,85.00000,150.0,27.0,2
1,Eaglecrest Ski Area,Alaska,2600.0,1540.000,1200.0,0.0,0.0,0.0,0.0,0.0,...,200.0,60.0,115.103943,44.0,156.5,47.000000,53.00000,90.0,0.0,2
2,Hilltop Ski Area,Alaska,2090.0,294.000,1796.0,0.0,0.0,0.0,0.0,0.0,...,30.0,30.0,115.103943,36.0,69.0,57.916957,64.16681,152.0,30.0,0
3,Arizona Snowbowl,Arizona,3127.5,961.125,1561.5,0.0,0.0,0.0,0.0,0.0,...,200.0,104.0,122.000000,81.0,260.0,57.916957,89.00000,122.0,0.0,2
4,Sunrise Park Resort,Arizona,3127.5,1800.000,1561.5,0.0,0.0,0.0,0.0,0.0,...,200.0,80.0,115.000000,49.0,250.0,74.000000,78.00000,104.0,27.0,2


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [81]:
# select object type columns but drop the name column 
ski_data_obj=ski_data.select_dtypes(include=['object']).drop(columns=['Name'])
# Concat the two dataframes and get dummies for the categorical variables
ski_data_dum = pd.concat([ski_data.drop(ski_data_obj, axis=1), pd.get_dummies(ski_data_obj)], axis=1)
ski_data_dum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 61 columns):
Name                    330 non-null object
summit_elev             330 non-null float64
vertical_drop           330 non-null float64
base_elev               330 non-null float64
trams                   330 non-null float64
fastEight               330 non-null float64
fastSixes               330 non-null float64
fastQuads               330 non-null float64
quad                    330 non-null float64
triple                  330 non-null float64
double                  330 non-null float64
surface                 330 non-null float64
total_chairs            330 non-null float64
Runs                    330 non-null float64
TerrainParks            330 non-null float64
LongestRun_mi           330 non-null float64
SkiableTerrain_ac       330 non-null float64
Snow Making_ac          330 non-null float64
daysOpenLastYear        330 non-null float64
yearsOpen               330 non-null floa

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [82]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = ski_data_dum.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y= ski_data_dum['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X)


## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [83]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [84]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [85]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred= model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [86]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
from sklearn.metrics import explained_variance_score, mean_absolute_error
# You can plug y_test and y_pred into the functions to evaluate the model
evs= explained_variance_score(y_test, y_pred)
evs

0.6005062094358982

In [87]:
mae= mean_absolute_error(y_test, y_pred)
mae

5.940476226748405

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [88]:
lm.intercept_


61.86583824084757

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [89]:
### Is it right that state dummies make up the bulk of the top 10 response variables?
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coeff= pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
# Convert the coeffiecients to absolute values
coeff= coeff.abs()
# Sort the values by Cofficient in dewcending order to view the response variables with largest impact 
coeff= coeff.sort_values(by='Coefficient', ascending=False)
coeff.head(10)


,Coefficient
trams,2.224546e+13
fastSixes,3.978081e+12
state_New York,1.943592e+12
quad,1.914827e+12
state_Michigan,1.834223e+12
fastQuads,1.789231e+12
state_Colorado,1.616057e+12
state_California,1.581462e+12
state_Pennsylvania,1.509131e+12
state_Wisconsin,1.391537e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [90]:
# Remove all the state dummy variables and save that to a special df for this model
ski_data_model2= ski_data_dum.iloc[:, :26]
# Check out the model2 df
ski_data_model2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 26 columns):
Name                 330 non-null object
summit_elev          330 non-null float64
vertical_drop        330 non-null float64
base_elev            330 non-null float64
trams                330 non-null float64
fastEight            330 non-null float64
fastSixes            330 non-null float64
fastQuads            330 non-null float64
quad                 330 non-null float64
triple               330 non-null float64
double               330 non-null float64
surface              330 non-null float64
total_chairs         330 non-null float64
Runs                 330 non-null float64
TerrainParks         330 non-null float64
LongestRun_mi        330 non-null float64
SkiableTerrain_ac    330 non-null float64
Snow Making_ac       330 non-null float64
daysOpenLastYear     330 non-null float64
yearsOpen            330 non-null float64
averageSnowfall      330 non-null float64
AdultWeekday  

In [93]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X2 = ski_data_model2.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y2= ski_data_model2['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler2 = preprocessing.StandardScaler().fit(X2)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X2_scaled=scaler2.transform(X2)

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y2 = y2.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.25, random_state=1)

lm = linear_model.LinearRegression()
model2 = lm.fit(X2_train,y2_train)

# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y2_pred= model2.predict(X2_test)

In [94]:
# You can plug y_test and y_pred into the functions to evaluate the model
# It looks like EVS decreased a biy after removing the state variables
evs2= explained_variance_score(y2_test, y2_pred)
evs2

0.5880419892392106

In [95]:
# It looks like MAE went up slightly when state variables were removed
mae2= mean_absolute_error(y2_test, y2_pred)
mae2

5.9738066373981225

In [96]:
# This is the average AdultWeekend price when accounting for all variables 
lm.intercept_

61.91364641351492

In [100]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coeff= pd.DataFrame(abs(lm.coef_), X2.columns, columns=['Coefficient'])
# Convert the coeffiecients to absolute values
coeff= coeff.abs()
# Sort the values by Cofficient in dewcending order to view the response variables with largest impact 
coeff= coeff.sort_values(by='Coefficient', ascending=False)
coeff.head(10)

,Coefficient
AdultWeekday,6.674839
LongestRun_mi,2.067108
total_chairs,1.738282
Snow Making_ac,0.816452
daysOpenLastYear,0.805400
base_elev,0.763218
averageSnowfall,0.733694
surface,0.452399
double,0.422685
SkiableTerrain_ac,0.359996


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

In [101]:
### I don't have summit_elev in the number 10 spot. I am assuming that has something to do with me using replacement
### for the outliers instead of removing them entirely 

#### Model 3

In [104]:
# Creating a new model without summit_elev and base_elev 
ski_data_model3= ski_data_model2.drop(columns=['summit_elev', 'base_elev'])
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X3 = ski_data_model3.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y3= ski_data_model3['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler3 = preprocessing.StandardScaler().fit(X3)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X3_scaled=scaler3.transform(X3)

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y3 = y3.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_scaled, y3, test_size=0.25, random_state=1)

lm = linear_model.LinearRegression()
model3 = lm.fit(X3_train,y3_train)

# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y3_pred= model3.predict(X3_test)

In [106]:
# You can plug y_test and y_pred into the functions to evaluate the model
evs3= explained_variance_score(y3_test, y3_pred)
evs3

0.6001980088362145

In [107]:
# It looks like MAE went up slightly when state variables were removed
mae3= mean_absolute_error(y3_test, y3_pred)
mae3

5.91205922656483

In [108]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
coeff= pd.DataFrame(abs(lm.coef_), X3.columns, columns=['Coefficient'])
# Convert the coeffiecients to absolute values
coeff= coeff.abs()
# Sort the values by Cofficient in dewcending order to view the response variables with largest impact 
coeff= coeff.sort_values(by='Coefficient', ascending=False)
coeff.head(10)

,Coefficient
AdultWeekday,6.618953
LongestRun_mi,2.084502
total_chairs,1.746368
Snow Making_ac,0.810174
daysOpenLastYear,0.803272
K-Means Clusters,0.771129
averageSnowfall,0.732763
surface,0.503409
double,0.423137
SkiableTerrain_ac,0.368313


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | evs |  |-|
| Model 2. | evs2| |'state'|
| Model 3. | evs3 | |'state','summit_elev','base_elev'|

In [110]:
data= [['Model 1', evs, mae, 'none'], ['Model 2', evs2, mae2, 'state'], ['Model 3', evs3, mae3, 'state, summit_elev, base_elev']]
models_df= pd.DataFrame(data, columns=['Model', 'Explained Variance', 'Mean Absolute Error', 'Features Dropped'])
models_df

,Model,Explained Variance,Mean Absolute Error,Features Dropped
0,Model 1,0.600506,5.940476,none
1,Model 2,0.588042,5.973807,state
2,Model 3,0.600198,5.912059,"state, summit_elev, base_elev"


Model Selection: Model 3 it has the 2nd best EVS and MAE but it has the most relevant columns in it